## Installing Required libraries
Step 1.1- Please refer to the "Technical Requirements" section in the book for the neccessary packages to be installed. Please note that chapter has differnt Pytorch Lightning version and thus diff torch dependancies. Some functions may not work with other versions than what is tested below, so please ensure correct versions.

In [1]:
#refer to book for correct version of package and import here
!pip install torch==1.10.0 torchvision==0.11.1 torchtext==0.11.0 torchaudio==0.10.0 --quiet
!pip install pytorch-lightning==1.5.2 --quiet

     |██████████████████████████████▎ | 834.1 MB 1.1 MB/s eta 0:00:43tcmalloc: large alloc 1147494400 bytes == 0x3a70c000 @  0x7f498fd3c615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |████████████████████████████████| 881.9 MB 16 kB/s 
     |████████████████████████████████| 23.3 MB 7.7 MB/s 
     |████████████████████████████████| 8.0 MB 41.0 MB/s 
     |████████████████████████████████| 2.9 MB 50.3 MB/s 
     |████████████████████████████████| 1.0 MB 8.6 MB/s 
     |████████████████████████████████| 419 kB 47.5 MB/s 
     |████████████████████████████████| 829 kB 49.8 MB/s 


## Step 1.2 Import the neccessary packages.
Please refer to the section, Importing the packages in the book for further details.

In [2]:
import torch
from torch import nn, optim
from torch.autograd import Variable
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from torch.utils.data import DataLoader
from torchmetrics.functional import accuracy

In [3]:
print("torch version:",torch.__version__)
print("pytorch ligthening version:",pl.__version__)

torch version: 1.10.0+cu102
pytorch ligthening version: 1.5.2


In [4]:
xor_input = [Variable(torch.Tensor([0, 0])),
           Variable(torch.Tensor([0, 1])),
           Variable(torch.Tensor([1, 0])),
           Variable(torch.Tensor([1, 1]))]

In [5]:
xor_target = [Variable(torch.Tensor([0])),
           Variable(torch.Tensor([1])),
           Variable(torch.Tensor([1])),
           Variable(torch.Tensor([0]))]

In [6]:
xor_data = list(zip(xor_input, xor_target))
train_loader = DataLoader(xor_data, batch_size=1000)

In [7]:
class XORModel(pl.LightningModule):
  def __init__(self):

    super(XORModel,self).__init__()
    
    self.input_layer = nn.Linear(2, 4)
    self.output_layer = nn.Linear(4,1)

    self.sigmoid = nn.Sigmoid()

    self.loss = nn.MSELoss()

  def forward(self, input):
    #print("INPUT:", input.shape)
    x = self.input_layer(input)
    #print("FIRST:", x.shape)
    x = self.sigmoid(x)
    #print("SECOND:", x.shape)
    output = self.output_layer(x)
    #print("THIRD:", output.shape)
    return output

  def configure_optimizers(self):
    params = self.parameters()
    optimizer = optim.Adam(params=params, lr = 0.01)
    return optimizer

  def training_step(self, batch, batch_idx):
    xor_input, xor_target = batch
    #print("XOR INPUT:", xor_input.shape)
    #print("XOR TARGET:", xor_target.shape)
    outputs = self(xor_input) 
    #print("XOR OUTPUT:", outputs.shape)
    loss = self.loss(outputs, xor_target)
    return loss 

In [8]:
from pytorch_lightning.utilities.types import TRAIN_DATALOADERS
checkpoint_callback = ModelCheckpoint()
model = XORModel()

trainer = pl.Trainer(max_epochs=500, callbacks=[checkpoint_callback])

trainer.fit(model, train_dataloaders=train_loader)

INFO:pytorch_lightning.utilities.distributed:GPU available: False, used: False
INFO:pytorch_lightning.utilities.distributed:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.distributed:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.callbacks.model_summary:
  | Name         | Type    | Params
-----------------------------------------
0 | input_layer  | Linear  | 12    
1 | output_layer | Linear  | 5     
2 | sigmoid      | Sigmoid | 0     
3 | loss         | MSELoss | 0     
-----------------------------------------
17        Trainable params
0         Non-trainable params
17        Total params
0.000     Total estimated model params size (MB)
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:407: UserWarning: The number of training samples (1) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  f"The number of trai

Training: 0it [00:00, ?it/s]

In [9]:
ls lightning_logs/

version_0/


In [10]:
print(checkpoint_callback.best_model_path)

/content/lightning_logs/version_0/checkpoints/epoch=499-step=499.ckpt


In [12]:
print(checkpoint_callback.best_model_path)
train_model = model.load_from_checkpoint(checkpoint_callback.best_model_path)
test = torch.utils.data.DataLoader(xor_input, batch_size=1)
for val in xor_input:
  _ = train_model(val)
  print([int(val[0]),int(val[1])], int(_.round()))

/content/lightning_logs/version_0/checkpoints/epoch=499-step=499.ckpt
[0, 0] 0
[0, 1] 1
[1, 0] 1
[1, 1] 0


In [13]:
print(checkpoint_callback.best_model_path)
train_model = model.load_from_checkpoint(checkpoint_callback.best_model_path)
total_accuracy = []
for xor_input, xor_target in train_loader:
  for i in range(100):
    output_tensor = train_model(xor_input)
    test_accuracy = accuracy(output_tensor, xor_target.int())
    total_accuracy.append(test_accuracy)
total_accuracy = torch.mean(torch.stack(total_accuracy))
print("TOTAL ACCURACY FOR 100 ITERATIONS: ", total_accuracy.item())

/content/lightning_logs/version_0/checkpoints/epoch=499-step=499.ckpt
TOTAL ACCURACY FOR 100 ITERATIONS:  1.0
